# ValyuContext

>[Valyu](https://www.valyu.network/) allows AI applications and agents to search the internet and proprietary data sources for relevant LLM ready information.

This notebook goes over how to use Valyu context tool in LangChain.

First, get an Valyu API key and add it as an environment variable. Get $10 free credit  by [signing up here](https://platform.valyu.network/).

## Overview

### Integration details
| Class                                                         | Package                                                        | Serializable | JS support |  Package latest |
|:--------------------------------------------------------------|:---------------------------------------------------------------| :---: | :---: | :---: |
| [ValyuSearchTool](https://github.com/valyu-network/langchain-valyu) | [langchain-valyu](https://pypi.org/project/langchain-valyu/) | ✅ | ❌  |  ![PyPI - Version](https://img.shields.io/pypi/v/langchain-valyu?style=flat-square&label=%20) |
| [ValyuContentsTool](https://github.com/valyu-network/langchain-valyu) | [langchain-valyu](https://pypi.org/project/langchain-valyu/) | ✅ | ❌  |  ![PyPI - Version](https://img.shields.io/pypi/v/langchain-valyu?style=flat-square&label=%20) |




## Setup

The integration lives in the `langchain-valyu` package.

In [ ]:
%pip install -qU langchain-valyu

In order to use the package, you will also need to set the `VALYU_API_KEY` environment variable to your Valyu API key.

In [ ]:
import getpass
import os

if not os.environ.get("VALYU_API_KEY"):
    os.environ["VALYU_API_KEY"] = getpass.getpass("Valyu API key:\n")

## Search Tool

### Instantiation

Here we show how to instantiate an instance of the Valyu search tool. This tool allows you to complete search queries using Valyu's Context API endpoint.


In [ ]:
from langchain_valyu import ValyuSearchTool

tool = ValyuSearchTool()

### Invocation

#### Invoke directly with args

The Valyu search tool accepts the following arguments during invocation:
- `query` (required): A natural language search query
- `search_type` (optional): Type of search: "all", "proprietary", or "web". Defaults to "all"
- `max_num_results` (optional): Maximum number of results to return (1-20). Defaults to 10
- `relevance_threshold` (optional): Minimum relevance score required for results (0.0-1.0). Defaults to 0.5
- `max_price` (optional): Maximum cost in dollars for this search. Defaults to 50.0
- `is_tool_call` (optional): Set to True when called by AI agents/tools. Defaults to True
- `start_date` (optional): Start date for time filtering in YYYY-MM-DD format
- `end_date` (optional): End date for time filtering in YYYY-MM-DD format
- `included_sources` (optional): List of URLs, domains, or datasets to include in search results
- `excluded_sources` (optional): List of URLs, domains, or datasets to exclude from search results
- `response_length` (optional): Content length per item: int for character count, or 'short' (25k), 'medium' (50k), 'large' (100k), 'max' (full content)
- `country_code` (optional): 2-letter ISO country code (e.g., 'GB', 'US') to bias search results to a specific country
- `fast_mode` (optional): Enable fast mode for faster but shorter results. Defaults to False

See the [Valyu API documentation](https://docs.valyu.network/overview) for the full documentation.


In [ ]:
search_results = tool._run(
    query="What are agentic search-enhanced large reasoning models?",
    search_type="all",
    max_num_results=5,
    relevance_threshold=0.4,
    is_tool_call=True,
    max_price=30.0,
)

print("Search Results:", search_results)

## Content Extraction Tool

The package also includes a `ValyuContentsTool` for extracting clean, structured content from web pages.

### Instantiation


In [ ]:
from langchain_valyu import ValyuContentsTool

contents_tool = ValyuContentsTool()

### Invocation

#### Invoke directly with args

The Valyu contents tool accepts the following arguments during invocation:
- `urls` (required): List of URLs to extract content from (maximum 10 URLs per request)

The tool can also be configured with additional parameters during instantiation:
- `summary` (optional): Enable content summarization. Can be `True`, `False`, a string instruction, or a dict with custom settings
- `extract_effort` (optional): Extraction effort level: "normal", "high", or "auto". Defaults to "normal"
- `response_length` (optional): Content length preference: int for character count, or "short" (25k), "medium" (50k), "large" (100k), "max" (full content). Defaults to "short"

For reliability and performance reasons, certain parameters may be required or restricted. See the [Valyu API documentation](https://docs.valyu.network/overview) for details.


In [ ]:
content_results = contents_tool._run(
    urls=["https://www.example.com/article1", "https://www.example.com/article2"]
)

print("Content Results:", content_results)

#### Configure with custom parameters

You can also instantiate the contents tool with custom parameters:


In [ ]:
# Configure with custom parameters
custom_contents_tool = ValyuContentsTool(
    summary=True,  # Enable automatic summarization
    extract_effort="high",  # Use high extraction effort
    response_length="medium",  # Medium length responses (50k chars)
)

# Extract content with custom configuration
custom_results = custom_contents_tool._run(
    urls=["https://www.example.com/long-article"]
)

print("Custom Results:", custom_results)

## Use within an agent

We can use our tools directly with an agent executor by binding the tool to the agent. This gives the agent the ability to dynamically set the available arguments to the Valyu search tool.


In [ ]:
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OPENAI_API_KEY:\n")

In [ ]:
# | output: false
# | echo: false

# !pip install -qU langchain langchain-openai
from langchain.chat_models import init_chat_model

llm = init_chat_model(model="gpt-4o", model_provider="openai", temperature=0)

In [ ]:
from langchain_valyu import ValyuSearchTool
from langgraph.prebuilt import create_react_agent

valyu_search_tool = ValyuSearchTool()

agent = create_react_agent(llm, [valyu_search_tool])

user_input = "What are the key factors driving recent stock market volatility, and how do macroeconomic indicators influence equity prices across different sectors?"

for step in agent.stream(
    {"messages": user_input},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

## API reference

For detailed documentation of all Valyu Context API features and configurations head to the API reference: https://docs.valyu.network/overview
